In [1]:
import warnings
import os
import sys
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
#os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
# sys.path.append(r"/mnt/dev/chexpert/CheXpert_Baseline_PyTorch/")

warnings.filterwarnings('ignore')

In [2]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
from fastai.vision import *
from torchvision.models import *
import yaml
import pandas as pd
import datetime
from sklearn.metrics import roc_auc_score, accuracy_score
from efficientnet_pytorch import model as enet
from pretrainedmodels import se_resnext101_32x4d, se_resnext50_32x4d, densenet121
from sklearn.metrics import confusion_matrix, mean_absolute_error


In [4]:
batch_size = 8
label_mat=3 
net_outdim = 1
bigpic=True
case = f'tmb_fold_batch_size_{batch_size}'


## dataset

In [6]:
from dataset.dataset_tmb import TCGADataset
dataset_train = TCGADataset(valid_fold=0, tmb_slice=['1','2'], patch_mode=True, expand_mod=False)
dl_train = DataLoader(dataset_train, batch_size=batch_size, shuffle=True, num_workers=4, drop_last=True,pin_memory=False)
dataset_val = TCGADataset(is_train=False, valid_fold=0, tmb_slice=['1','2'], patch_mode=True, expand_mod=False)
dl_val = DataLoader(dataset_val, batch_size=batch_size, shuffle=False, num_workers=4,pin_memory=False,drop_last=False)

dataset_distile = TCGADataset(expand_mod=True)
dl_distile = DataLoader(dataset_distile, batch_size=batch_size*2, shuffle=False, num_workers=0, drop_last=False,pin_memory=False)

data = DataBunch(train_dl=dl_train, valid_dl=dl_val, test_dl=dl_distile, device='cuda', no_check=True)

## model

In [7]:
class enetv2(nn.Module):
    def __init__(self, backbone, out_dim, patch_num, batch_size):
        super(enetv2, self).__init__()
        self.enet = enet.EfficientNet.from_pretrained(backbone)
        self.last_channel_size = self.enet._fc.in_features
        self.myfc = nn.Linear(self.last_channel_size, out_dim)
        del self.enet._fc
        self.patch_num = patch_num
        self.batch_size = batch_size

    def forward(self, x):
        bs=x.shape[0]
#         x = x.reshape((bs*self.patch_num, 3, self.img_size, self.img_size))
#         print(x.shape)
        x = self.enet.extract_features(x)
        x = self.enet._avg_pooling(x)
#         print(x.shape)
        x = x.view(bs, self.last_channel_size)
        x = self.enet._dropout(x)
#         x = x.reshape((bs, self.last_channel_size))
        x = self.myfc(x)
        return x

In [8]:
enet_type = 'efficientnet-b0'
model = enetv2(enet_type, net_outdim, 16, batch_size).cuda()


Loaded pretrained weights for efficientnet-b0


In [25]:
class LearnCallbacks(LearnerCallback):
    _order = 99
    def __init__(self, learn, save_case=''):
        super().__init__(learn)
        self.epoch = 0
        self.skip = False
        self.skip_validate = False
        self.state = {"skip_validate":False}
        self.learn = learn
        self.save_case = save_case

    def on_train_begin(self,**kwargs):
        pass
    def on_epoch_begin(self, **kwargs):
        print(f'lr_{self.learn.opt.lr}\twd_{self.learn.opt.wd}')

    def on_epoch_end(self,n_epochs,**kwargs):
        self.epoch += 1
#         self.skip_val(n_epochs)
        self.save()
        return self.state
    
    def skip_val(self, n_epochs):
        self.skip_validate = ~self.skip_validate
        if self.epoch+1 == n_epochs:self.skip_validate=False
        self.state.update({"skip_validate":self.skip_validate})
        
    def save(self):
        try:
            n=self.learn.recorder.metrics[-1][-1].item()
        except:
            n=0.0
        learn.save(f'{self.save_case} {datetime.datetime.now():%Y-%m-%d %H:%M} metrics:{n:.3}')
        print(f"model saved: {self.save_case} {datetime.datetime.now():%Y-%m-%d %H:%M} metrics:{n:.3}")
    
    def save_info(self):
        pass
class acc_ad(Callback):
    pass
class acc_sc(Callback):
    pass
class acc_all(Callback):
    def __init__(self):
        super().__init__()
        self.esp = 1e-8

    def on_epoch_begin(self,**kwargs):
        self.x_t = []
        self.y_t = []
        self.log_loss = [] 
        
    def on_batch_end(self, last_output, last_target, **kwargs):
        self.x_t.append(last_output.detach().flatten())#.sigmoid()
        self.y_t.append(last_target.flatten())  
#         pred = logits.sigmoid().sum(1).detach().round()
#         print(self.xo, self.yo)
        
    def on_epoch_end(self, epoch, last_metrics, **kwargs):        
        self.last_metrics = last_metrics
        self.y_tmb = torch.cat(self.x_t).detach().cpu()
        self.l_tmb = torch.cat(self.y_t).detach().cpu()
        f1_tmb = mean_absolute_error(self.l_tmb[self.l_tmb>0], self.y_tmb[self.l_tmb>0])
        print(f1_tmb)
#         print(self.y_tmb, self.l_tmb)
        f1_tmb = roc_auc_score(self.l_tmb>0, self.y_tmb)
#         print((self.y_tmb>=0).sum(), (self.l_tmb>0).sum())
        print(f'tmb mae:{f1_tmb}')
        

        return {"last_metrics":self.last_metrics + [f1_tmb]}
    

In [10]:
learn = Learner(data, model, loss_func=nn.SmoothL1Loss(), metrics=[acc_all()])
learn.callbacks=[LearnCallbacks(learn, case)]

In [26]:
learn.metrics=[acc_all()]

In [27]:
lr =1e-4
learn.fit_one_cycle(30,lr)

epoch,train_loss,valid_loss,acc_all,time
0,0.126361,0.160123,0.564008,02:06
1,0.140742,0.154847,0.565737,02:06
2,0.137070,0.159931,0.562406,02:05
3,0.146474,0.156272,0.548077,02:05
4,0.158674,0.157169,0.559138,02:04
5,0.153612,0.162277,0.550339,02:07
6,0.131164,0.159572,0.591315,02:04
7,0.143125,0.162633,0.536608,02:05
8,0.149033,0.166865,0.580631,02:04
9,0.147575,0.161163,0.539153,02:04


lr_4e-06	wd_0.01
1.0008072
tmb mae:0.5640082956259427
model saved: tmb_fold_batch_size_8 2020-06-30 22:48 metrics:0.564
lr_6.894754202276402e-06	wd_0.01
1.0107304
tmb mae:0.5657365510306687
model saved: tmb_fold_batch_size_8 2020-06-30 22:50 metrics:0.566
lr_1.5229866730289061e-05	wd_0.01
1.0206956
tmb mae:0.5624057315233786
model saved: tmb_fold_batch_size_8 2020-06-30 22:52 metrics:0.562
lr_2.8000000000000003e-05	wd_0.01
1.0595853
tmb mae:0.548076923076923
model saved: tmb_fold_batch_size_8 2020-06-30 22:55 metrics:0.548
lr_4.366488747198734e-05	wd_0.01
0.96322906
tmb mae:0.5591377576671694
model saved: tmb_fold_batch_size_8 2020-06-30 22:57 metrics:0.559
lr_6.0335112528012655e-05	wd_0.01
1.0666593
tmb mae:0.550339366515837
model saved: tmb_fold_batch_size_8 2020-06-30 22:59 metrics:0.55
lr_7.599999999999999e-05	wd_0.01
1.0190051
tmb mae:0.5913147310206134
model saved: tmb_fold_batch_size_8 2020-06-30 23:01 metrics:0.591
lr_8.877013326971095e-05	wd_0.01
1.0634651
tmb mae:0.5366075917

In [28]:
lr =1e-3
learn.fit_one_cycle(30,lr)

epoch,train_loss,valid_loss,acc_all,time
0,0.123084,0.182711,0.578494,02:07
1,0.125820,0.170253,0.566114,02:05
2,0.132300,0.215616,0.584873,02:04
3,0.138920,0.166228,0.557190,02:03
4,0.168905,0.208532,0.423423,02:04
5,0.152864,0.150356,0.501225,02:05
6,0.163718,0.148282,0.604858,02:06
7,0.170887,0.148191,0.468043,02:04
8,0.184885,0.166862,0.552696,02:03
9,0.176346,0.153737,0.438631,02:04


lr_4e-05	wd_0.01
1.0383235
tmb mae:0.5784942182001006
model saved: tmb_fold_batch_size_8 2020-07-01 10:26 metrics:0.578
lr_6.8947542022764e-05	wd_0.01
1.02005
tmb mae:0.5661136249371543
model saved: tmb_fold_batch_size_8 2020-07-01 10:28 metrics:0.566
lr_0.0001522986673028906	wd_0.01
0.9063783
tmb mae:0.5848730517848165
model saved: tmb_fold_batch_size_8 2020-07-01 10:30 metrics:0.585
lr_0.00028	wd_0.01
0.97895247
tmb mae:0.5571895424836601
model saved: tmb_fold_batch_size_8 2020-07-01 10:32 metrics:0.557
lr_0.00043664887471987334	wd_0.01
1.4193715
tmb mae:0.4234225741578683
model saved: tmb_fold_batch_size_8 2020-07-01 10:34 metrics:0.423
lr_0.0006033511252801266	wd_0.01
1.1270839
tmb mae:0.5012254901960784
model saved: tmb_fold_batch_size_8 2020-07-01 10:36 metrics:0.501
lr_0.0007599999999999999	wd_0.01
1.1310614
tmb mae:0.604857968828557
model saved: tmb_fold_batch_size_8 2020-07-01 10:38 metrics:0.605
lr_0.0008877013326971094	wd_0.01
1.1102203
tmb mae:0.46804298642533937
model save

In [22]:
learn.validate(metrics=[acc_all()])

tensor([-5.1393e-01, -1.1359e+00, -5.3170e-01, -6.0351e-01, -2.2838e-01,
        -1.8872e-01, -3.4723e-01, -3.0909e-01, -4.1755e-01, -3.1941e-01,
         1.4670e-02, -7.6058e-02, -1.8347e-01, -4.8396e-01, -4.7792e-01,
        -6.7216e-01, -3.3592e-01, -5.9198e-01, -3.9655e-01, -2.8151e-01,
        -3.3516e-01, -2.2147e-01, -3.5040e-01, -5.4543e-01, -2.0055e-01,
        -5.1220e-01, -2.8122e-01, -3.1906e-01, -3.3232e-01, -7.2321e-02,
        -1.1532e-01, -8.1220e-01, -3.3024e-01, -3.0315e-01, -1.6136e-01,
        -4.2075e-01, -2.7983e-01, -9.0099e-01, -2.3155e-01, -5.9525e-01,
        -2.8585e-01,  1.3763e-01, -6.7996e-01, -3.8177e-01, -6.9915e-01,
        -6.8695e-01, -3.7221e-01, -5.3371e-01, -2.8220e-01, -1.7869e-01,
        -2.3581e-01, -4.3992e-01, -2.9468e-01, -7.2156e-01, -1.3517e-01,
        -5.7974e-01, -1.4961e-01, -4.3119e-01, -4.5308e-01, -5.8438e-01,
        -7.2349e-01, -4.9823e-01, -6.5122e-01, -4.2443e-01, -5.6604e-01,
        -4.4913e-02, -2.6458e-01, -5.5820e-01, -6.0

[0.14996693, 0.581856460532931]

In [ ]:
lr =1e-4
learn.fit_one_cycle(30,lr,pct_start=0, final_div=1e2, wd=1e-4)

epoch,train_loss,valid_loss,acc_all,time
0,0.282936,0.703336,0.594614,02:07
1,0.323030,0.646773,0.635087,01:50
2,0.289501,0.688594,0.573749,01:49
3,0.283212,0.730725,0.572587,01:47
4,0.269531,0.746019,0.594237,01:47
5,0.269379,0.719967,0.593294,01:47
6,0.250671,0.674989,0.621198,01:48
7,0.266391,0.707985,0.605738,01:48
8,0.282604,0.603642,0.648159,01:46
9,0.287259,0.601695,0.668583,01:46


lr_4e-06	wd_0.0001
tmb mae:0.594614127702363
model saved: tmb_fold_batch_size_8 2020-06-28 09:47 metrics:0.595
lr_9.973157730520429e-05	wd_0.0001
tmb mae:0.6350867269984918
model saved: tmb_fold_batch_size_8 2020-06-28 09:49 metrics:0.635
lr_9.892377014066981e-05	wd_0.0001
tmb mae:0.5737493715434892
model saved: tmb_fold_batch_size_8 2020-06-28 09:50 metrics:0.574
lr_9.758542201714877e-05	wd_0.0001
tmb mae:0.5725867269984917
model saved: tmb_fold_batch_size_8 2020-06-28 09:52 metrics:0.573
lr_9.573119615674986e-05	wd_0.0001
tmb mae:0.5942370537958773
model saved: tmb_fold_batch_size_8 2020-06-28 09:54 metrics:0.594
lr_9.338140784602127e-05	wd_0.0001
tmb mae:0.5932943690296631
model saved: tmb_fold_batch_size_8 2020-06-28 09:56 metrics:0.593
lr_9.056180185742019e-05	wd_0.0001
tmb mae:0.6211978381096028
model saved: tmb_fold_batch_size_8 2020-06-28 09:58 metrics:0.621
lr_8.730327038419821e-05	wd_0.0001
tmb mae:0.6057378079436904
model saved: tmb_fold_batch_size_8 2020-06-28 09:59 metrics

In [15]:
lr =1e-3
learn.fit_one_cycle(100,lr,pct_start=0, final_div=1e3, wd=1e-4)

epoch,train_loss,valid_loss,acc_all,time
0,3.689133,3.391091,3.840389,05:02
1,3.898280,3.377273,3.828476,04:46
2,3.723307,3.344445,3.799606,04:44
3,3.609624,3.482748,3.935591,04:45
4,3.791501,3.393774,3.843707,04:44
5,3.659469,3.365929,3.817632,04:45
6,3.712448,3.373162,3.824712,04:45
7,3.563741,3.389116,3.839618,04:45
8,3.625133,3.398711,3.847990,04:45
9,3.794913,3.384864,3.836459,04:45


lr_4e-05	wd_0.0001
tmb mae:3.840388774871826
model saved: tmb_fold_batch_size_8 2020-06-18 22:28 metrics:3.84
lr_0.0009997555594960068	wd_0.0001
tmb mae:3.8284761905670166
model saved: tmb_fold_batch_size_8 2020-06-18 22:33 metrics:3.83
lr_0.0009990184182396022	wd_0.0001
tmb mae:3.7996063232421875
model saved: tmb_fold_batch_size_8 2020-06-18 22:37 metrics:3.8
lr_0.000997789299491253	wd_0.0001
tmb mae:3.935591459274292
model saved: tmb_fold_batch_size_8 2020-06-18 22:42 metrics:3.94
lr_0.000996069416242771	wd_0.0001
tmb mae:3.8437066078186035
model saved: tmb_fold_batch_size_8 2020-06-18 22:47 metrics:3.84
lr_0.0009938604658112778	wd_0.0001
tmb mae:3.817631721496582
model saved: tmb_fold_batch_size_8 2020-06-18 22:52 metrics:3.82
lr_0.00099116462816416	wd_0.0001
tmb mae:3.824711799621582
model saved: tmb_fold_batch_size_8 2020-06-18 22:56 metrics:3.82
lr_0.0009879845637677024	wd_0.0001
tmb mae:3.8396177291870117
model saved: tmb_fold_batch_size_8 2020-06-18 23:01 metrics:3.84
lr_0.0009

tmb mae:3.8249852657318115
model saved: tmb_fold_batch_size_8 2020-06-19 03:37 metrics:3.82
lr_0.000259920865070526	wd_0.0001
tmb mae:3.8370258808135986
model saved: tmb_fold_batch_size_8 2020-06-19 03:42 metrics:3.84
lr_0.0002462896289570466	wd_0.0001
tmb mae:3.841529369354248
model saved: tmb_fold_batch_size_8 2020-06-19 03:47 metrics:3.84
lr_0.00023290926778859658	wd_0.0001
tmb mae:3.820585250854492
model saved: tmb_fold_batch_size_8 2020-06-19 03:52 metrics:3.82
lr_0.0002197929863662184	wd_0.0001
tmb mae:3.825089931488037
model saved: tmb_fold_batch_size_8 2020-06-19 03:57 metrics:3.83
lr_0.00020695372887612852	wd_0.0001
tmb mae:3.8338708877563477
model saved: tmb_fold_batch_size_8 2020-06-19 04:01 metrics:3.83
lr_0.00019440416611536692	wd_0.0001
tmb mae:3.8250904083251953
model saved: tmb_fold_batch_size_8 2020-06-19 04:06 metrics:3.83
lr_0.0001821566829872517	wd_0.0001
tmb mae:3.829796075820923
model saved: tmb_fold_batch_size_8 2020-06-19 04:11 metrics:3.83
lr_0.0001702233662789

## patch to whole

In [138]:
class tcga_whole_dataset():
    def __init__(self, learn, dataset):
        super().__init__()
        self.dataset =  dataset
        self.df = dataset.df
        self.fns = list(set(self.df['FILE_NAME'].tolist()))
        self.model = learn.model.eval()
        
    def __getitem__(self, i):
        df = self.df[self.df.FILE_NAME == self.fns[i]]
        n = df.shape[0]
        features_sum = 0
        with torch.no_grad():
            for i_patch, row in df.iterrows():
                img, label = self.dataset[i_patch]
                img = torch.tensor(img).unsqueeze(0).cuda()
                feature = self.model.enet.extract_features(img)
                features_sum += feature
            tmb = np.float32(row.TMB_38Mb)
            feature_average = features_sum / n
        return feature_average.squeeze(0), (label.argmax().astype(np.long), tmb)
    def __len__(self):
#         return 8
        return len(self.fns)

In [139]:
class simple_model(nn.Module):
    def __init__(self, num_input_features, ):
        super().__init__()
        self.out_put_dim = num_input_features//2
        self.m = nn.Sequential(OrderedDict(
        {'norm1': nn.BatchNorm2d(num_input_features),
        'relu1': nn.ReLU(inplace=False),
        'conv1': nn.Conv2d(num_input_features, self.out_put_dim, kernel_size=3, stride=1,
                                           bias=False),
        'norm2': nn.BatchNorm2d(self.out_put_dim),
        'relu2': nn.ReLU(inplace=False),
        'conv2': nn.Conv2d(self.out_put_dim, self.out_put_dim, kernel_size=3, stride=1,
                                           bias=False),
        'norm3': nn.BatchNorm2d(self.out_put_dim),
        'relu3': nn.ReLU(inplace=False),
        'conv3': nn.Conv2d(self.out_put_dim, self.out_put_dim, kernel_size=3, stride=1,
                                           bias=False)}))
        self.pooling = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Linear(num_input_features//2, 3)
        self.tmb_fc = nn.Linear(num_input_features//2, 1)

    def forward(self, x):
        x = self.m(x)
        x = self.pooling(x)
        x = x.view(x.shape[0], self.out_put_dim)
        x1 = self.fc(x)
        x2 = self.tmb_fc(x)
        return x1, x2

In [140]:
w_ds = tcga_whole_dataset(learn, data.valid_ds)
model_whole = simple_model(1280).cuda()
batch_size_whole = 8
learn.load('/mnt/data/tcga/models/bigpic_True_label_3fold_batch_size_8 2020-06-17 10:40 acc:0.813')

Learner(data=DataBunch;

Train: <dataset.tcga_dataset.TCGADataset object at 0x7fb197b421d0>;

Valid: <dataset.tcga_dataset.TCGADataset object at 0x7fb18d6ecb00>;

Test: <dataset.tcga_dataset.TCGADataset object at 0x7fb18d586a90>, model=enetv2(
  (enet): EfficientNet(
    (_conv_stem): Conv2dStaticSamePadding(
      3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False
      (static_padding): ZeroPad2d(padding=(0, 1, 0, 1), value=0.0)
    )
    (_bn0): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
    (_blocks): ModuleList(
      (0): MBConvBlock(
        (_depthwise_conv): Conv2dStaticSamePadding(
          32, 32, kernel_size=(3, 3), stride=[1, 1], groups=32, bias=False
          (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
        )
        (_bn1): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
        (_se_reduce): Conv2dStaticSamePadding(
          32, 8, kernel_size=

In [141]:
dataset_train_whole = tcga_whole_dataset(learn, data.train_ds)
dl_train_whole = DataLoader(dataset_train_whole, batch_size=batch_size_whole, shuffle=True, num_workers=0, drop_last=True,pin_memory=False)

dataset_val_whole = tcga_whole_dataset(learn, data.valid_ds)
dl_val_whole = DataLoader(dataset_val_whole, batch_size=batch_size_whole, shuffle=False, num_workers=0, pin_memory=False,drop_last=False)

data_whole = DataBunch(train_dl=dl_train_whole, valid_dl=dl_val_whole, device='cuda', no_check=True)

In [142]:
class acc_whole(Callback):
    def __init__(self):
        super().__init__()
        self.esp = 1e-8

    def on_epoch_begin(self,**kwargs):
        self.xo = []
        self.yo = []
        self.x_t = []
        self.y_t = []
        
    def on_batch_end(self, last_output, last_target, **kwargs):
        self.x_t.append(last_output[1].detach().flatten())
        self.y_t.append(last_target[1].flatten())  
        self.xo.append(last_output[0].argmax(-1).detach().flatten())
        self.yo.append(last_target[0].flatten())
#         pred = logits.sigmoid().sum(1).detach().round()
#         print(self.xo, self.yo)
        
    def on_epoch_end(self, epoch, last_metrics, **kwargs):        
        self.last_metrics = last_metrics
        self.y = np.array(torch.cat(self.xo).detach().cpu().numpy())
        self.l = np.array(torch.cat(self.yo).detach().cpu().numpy())
#         self.y_tmb = np.array(torch.cat(self.x_t).detach().cpu().numpy())
#         self.l_tmb = np.array(torch.cat(self.y_t).detach().cpu().numpy())  
        self.y_tmb = torch.cat(self.x_t).detach().cpu()
        self.l_tmb = torch.cat(self.y_t).detach().cpu()
        f1_tmb = mean_absolute_error(self.l_tmb[self.l_tmb>0], self.y_tmb[self.l_tmb>0])
#         print((self.y_tmb>=0).sum(), (self.l_tmb>0).sum())
        print(f'tmb mae:{f1_tmb}')
        matrix = confusion_matrix(self.y, self.l)
        print(matrix)
        return {"last_metrics":self.last_metrics + [(self.y==self.l).sum()/self.y.shape[0]]}

In [144]:
learn_whole = Learner(data_whole, model_whole, loss_func=tmb_cls_loss(), callback_fns=[LearnCallbacks], metrics=[acc_whole()])

In [ ]:
learn_whole.fit_one_cycle(15, wd=1e-3)

epoch,train_loss,valid_loss,acc_whole,time
0,4.434596,4.266017,0.795527,26:29
1,4.155612,3.937129,0.857827,27:41
2,4.524951,4.193159,0.856230,28:43
3,4.706205,4.411701,0.865815,26:55
4,4.173303,4.275243,0.861022,26:38


lr_0.00012	wd_0.001
torch.Size([626]) torch.Size([626])
tmb mae:3.9100501537323
[[108  22   7]
 [  1 154  12]
 [ 12  74 236]]
model saved: bigpic_True_label_3fold_batch_size_8 2020-06-18 14:44 acc:0.796
lr_0.0004568960019086716	wd_0.001
torch.Size([626]) torch.Size([626])
tmb mae:3.8521065711975098
[[108  10   5]
 [  6 220  41]
 [  7  20 209]]
model saved: bigpic_True_label_3fold_batch_size_8 2020-06-18 15:12 acc:0.858
lr_0.00130994662415962	wd_0.001
torch.Size([626]) torch.Size([626])
tmb mae:4.201976776123047
[[110   7   4]
 [  2 192  17]
 [  9  51 234]]
model saved: bigpic_True_label_3fold_batch_size_8 2020-06-18 15:41 acc:0.856
lr_0.00228	wd_0.001
torch.Size([626]) torch.Size([626])
tmb mae:4.1128315925598145
[[ 99   1   1]
 [  6 221  32]
 [ 16  28 222]]
model saved: bigpic_True_label_3fold_batch_size_8 2020-06-18 16:08 acc:0.866
lr_0.002913157373931708	wd_0.001
torch.Size([626]) torch.Size([626])
tmb mae:4.171572685241699
[[109   4   6]
 [  4 201  20]
 [  8  45 229]]
model saved: 

In [102]:
learn_whole.fit_one_cycle(15, wd=1e-3)

AssertionError: Your training dataloader is empty, can't train a model.
        Use a smaller batch size (batch size=8 for 4 elements).

In [134]:
learn_whole.validate()

torch.Size([8]) torch.Size([8])
tensor(8) tensor(8)
tmb mae:5.486178398132324
[[0 0 0]
 [1 1 0]
 [2 0 4]]


[6.0888753, 0.625]

## Distile

In [16]:
class Distile(LearnerCallback):
    _order = 99
    def __init__(self, learn):
        super().__init__(learn)
        self.learn = learn
        self.df = learn.data.test_ds.df
        self.data_dir = learn.data.test_ds.data_dir
        
    def on_epoch_begin(self,**kwargs):
        self.xo = []
        self.yo = []

    def on_batch_end(self, last_output, last_target, **kwargs):
        self.xo.append(last_output.softmax(-1).detach())
        self.yo.append(last_target)
#         pred = logits.sigmoid().sum(1).detach().round()
#         print(self.xo[0].shape, self.yo[0].shape)
        
    def on_epoch_end(self, epoch, last_metrics, **kwargs):        
        self.y = np.array(torch.cat(self.xo, dim=0).detach().cpu().numpy())
        self.l = np.array(torch.cat(self.yo, dim=0).detach().cpu().numpy())
        new_label = (self.y + self.l) / 2
        print(self.df.shape, new_label.shape)
        self.df['new_label3'] = self.df['new_label']
        self.df['new_label'] = new_label.tolist()
        self.df.to_csv(os.path.join(self.data_dir, 'train_distile2.csv'), index=None, sep='\t')


In [17]:
distile = Distile(learn)
learn.validate(dl=data.test_dl, metrics=[acc_all()], callbacks=[distile])

[[ 776    9  116]
 [  48 5610  543]
 [  30  282 5053]]
(12467, 20) (12467, 3)


[0.37560260491476893, 0.9175423117028957]

In [18]:
learn.data = data
learn.validate(dl=data.valid_dl, metrics=[acc_all()])

[[ 258    7   17]
 [ 194 1111  367]
 [ 116  209 1437]]


[0.5047397499641382, 0.7551130247578041]

In [ ]:
lr =5e-6
learn.fit_one_cycle(20,lr,pct_start=0.1, final_div=1e2)

In [ ]:
learn.metrics = [metrics_callback(dataset_val.df)]
learn.validate()

In [42]:
with open('ori_label.pkl', 'wb') as f:
    pickle.dump(dcp, f)